<a href="https://colab.research.google.com/github/tohver/Natural_Language_Processing/blob/master/fastai3_Sentiment_Oldschool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastai==1.0

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.text import *

In [ ]:
from fastai.utils.mem import GPUMemTrace

In [ ]:
import sklearn.feature_extraction.text as sklearn_text
import pickle

In [ ]:
?? URLs

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/root/.fastai/data/imdb_sample')

In [ ]:
import os
os.listdir(path) 

['texts.csv']

In [ ]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [ ]:
%%time

# throws `BrokenProcessPool' Error sometimes. Keep trying `till it works!

count = 0
error = True
while error:
  try:
    #preprocess:
    movie_reviews = (TextList.from_csv(path, 'texts.csv', cols = 'text').split_from_df(col = 2)
      .label_from_df(cols = 0))
    error = False
    print(f'failure count is {count}\n')
  except: #catch all exceptions
    # accumulate failure count
    count = count +1
    print(f'failure count is {count}')


In [ ]:
movie_reviews.train.x[0]

In [ ]:
movie_reviews.train.x[0].data

# Examine the movie_reviews object 

In [ ]:
dir(movie_reviews)

In [ ]:
len(movie_reviews.train)

In [ ]:
movie_reviews.train[6]

In [ ]:
print(len(movie_reviews.valid))
print(movie_reviews.valid[6])

In [ ]:
movie_reviews.train.x[3:6]

## Examine the structure of training set

In [ ]:
movie_reviews.train.x[:3]

In [ ]:
movie_reviews.train.y[:3]

In [ ]:
chars = movie_reviews.train.x[0].text
print(f'chars: {chars}')

In [ ]:
print(f'1st review has {len(chars)} characters and {len(chars.split())} words')

### tokens
mapped to integers (numericalized)

In [ ]:
movie_reviews.train.x[3].data

In [ ]:
movie_reviews.train.x[3]

In [ ]:
print(movie_reviews.train.x[3].text, '\n')

# IMDB Vocabulary
**movie_reviews object** has **.vocab property**

In [ ]:
movie_reviews.vocab

In [ ]:
# vocabs object: dictionary, tokens : their integer representation
# 2 metods:
# stoi (string to index) - dictionary (defaultdict)
# itos (index to string) - list

In [ ]:
# test:
print(movie_reviews.vocab.stoi['language'])
print(movie_reviews.vocab.itos[917])

In [ ]:
movie_reviews.vocab.stoi['blabla']

In [ ]:
movie_reviews.vocab.itos[movie_reviews.vocab.stoi['of']] # taki zarcik

In [ ]:
movie_reviews.vocab.itos[1000:1010]

In [ ]:
type(movie_reviews.vocab.itos)
# list, moze nie najlepszy wybor, ale ok
# return: token z danym indexem
# podejrzewam ze po prostu zwraca list[nr_indexu]

#### są uporządkowane wg frequency

In [ ]:
type(movie_reviews.vocab.stoi)

In [ ]:
movie_reviews.vocab.stoi

In [ ]:
movie_reviews.vocab.itos

In [ ]:
print(f'itos: {len(movie_reviews.vocab.itos)}')
print(f'stoi: {len(movie_reviews.vocab.stoi)}')

#### defaultdict

**In a defaultdict, rare words that appear fewer than three times in the corpus, and words that are not in the dictionary, are mapped to a default value, in this case, zero**  
many words map to unknow

In [ ]:
from collections import defaultdict
icecream = defaultdict(lambda: 'vanilla')

In [ ]:
icecream['a'] = '123'
icecream['b'] = '456'

In [ ]:
icecream['bubu']

In [ ]:
movie_reviews.vocab.stoi['makers']

In [ ]:
movie_reviews.vocab.stoi['anodyne']

In [ ]:
movie_reviews.vocab.itos[0]

In [ ]:
welcheUnknown = [word for word, num in movie_reviews.vocab.stoi.items() if num == 0]
len(welcheUnknown)

In [ ]:
print(f'len(stoi) - len(itos) = {len(movie_reviews.vocab.stoi) - len(movie_reviews.vocab.itos)}')
print('The difference: xxunk')

# 3. Map the movie rewiews into the vector space

In [ ]:
print(f'number of unique tokens in the reviews: {len(movie_reviews.vocab.itos)}')


### Counter
A **Counter** is a useful Python object.  A **Counter** applied to a list returns an ordered dictionary whose keys are the unique elements in the list, and whose values are the counts of the unique elements. Counters are from the collections module (along with OrderedDict, defaultdict, deque, and namedtuple).
Here is how Counters work:

In [ ]:
# parametr: reviewIndex - index of review in dataset (one of the 800 data)
TokenCounter = lambda reviewIndex: Counter(movie_reviews.train.x[reviewIndex].data)
TokenCounter(0).items()

In [ ]:
movie_reviews.train.x[456].data

In [ ]:
TokenCounter(546).items()

In [ ]:
# Review #33:
len(TokenCounter(33))

Convert a matrix to CSR format (Compressed Sparse Row)

## BIG O:
Convertions between CSR & CSC are linear time operations

In [ ]:
# document-term matrix in CSR format
# return (values, columnIndices, rowPointer)

def get_doc_term_matrix(text_list, n_terms):
  '''
  input: text_list: a TextList object
  n_terms: the number of tokens in IMDB vocabulary

  output: 
  CSR format sparse representation of the document-term matrix, 
  as scipy.sparse.csr_matrix object
  '''

  # initialize arrays
  values = []
  column_indices = []
  row_pointer = []
  row_pointer.append(0)

  # from the TextListObject:
  for _, doc in enumerate (text_list):
    feature_counter = Counter(doc.data) # .data: list of token-indices in the revies (patrz wyżej, TextList object)
      # movie_reviews.train.x[3].data
    column_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    # tack on N (number of nonzero elements in the matrix) to the end of the row_pointer array
    row_pointer.append(len(values))

  return scipy.sparse.csr_matrix((values, column_indices, row_pointer),
                                 shape = (len(row_pointer) -1, n_terms),
                                 dtype = int
                                 )

# czyli podajemy do SciPi w formacie COO, on konwertuje dalej



In [ ]:
movie_reviews.train[0]

In [ ]:
%%time
train_doc_term = get_doc_term_matrix(movie_reviews.train.x, len(movie_reviews.vocab.itos))

In [ ]:
print(type(train_doc_term))
train_doc_term.shape

# Embedding 

#### Make a `count_vectorizer` function that represents a movie review as a 6008-dimensional `embedding vector`
#### The `indices` of  the `embedding vector` correspond to the n6008 numericalized tokens in the vocabulary; the `values` specify how often the corresponding token appears in the review. 

In [ ]:
nTerms = len(movie_reviews.vocab.itos)
nDocs = len(movie_reviews.train.x)
makeTokenCounter = lambda reviewIndex: Counter(movie_reviews.train.x[reviewIndex].data)

def countVectorizer(reviewIndex, nTerms = nTerms, nmakeTokenCounter = makeTokenCounter):
  '''
  input: review index, nTerms, tokenizer function
  output: embeding vector for the review
  '''
  embeddingVector = np.zeros(nTerms)
  keys = list(makeTokenCounter(reviewIndex).keys())
  values = list(makeTokenCounter(reviewIndex).values())
  embeddingVector[keys] = values
  return embeddingVector

# embedding vector for the 1st review:
embeddingVector = countVectorizer(0)  

In [ ]:
print(f'the review is embedded in {len(embeddingVector)} dimensional vector')
embeddingVector

# 4. Create the document-term matrix for the IMDb

#### In non-deep learning methods of NLP, we are often interested only in `which words` were used in a review, and `how often each word got used`. This is known as the `bag of words` approach, and it suggests a really simple way to store a document (in this case, a movie review). 

#### For each review we can keep track of which words were used and how often each word was used with a `vector` whose `length` is the number of tokens in the vocabulary, which we will call `n`. The `indexes` of this `vector` correspond to the `tokens` in the `IMDb vocabulary`, and the`values` of the vector are the number of times the corresponding tokens appeared in the review. For example the values stored at indexes 0, 1, 2, 3, 4 of the vector record the number of times the 5 tokens ['xxunk','xxpad','xxbos','xxeos','xxfld'] appeared in the review, respectively.

#### Now, if our movie review database has `m` reviews, and each review is represented by a `vector` of length `n`, then vertically stacking the row vectors for all the reviews creates a matrix representation of the IMDb, which we call its `document-term matrix`. The `rows` correspond to `documents` (reviews), while the `columns` correspond to `terms` (or tokens in the vocabulary).

In the previous lesson, we used [sklearn's CountVectorizer](https://github.com/scikit-learn/scikit-learn/blob/55bf5d9/sklearn/feature_extraction/text.py#L940) to generate the `vectors` that represent individual reviews. Today we will create our own (similar) version.  This is for two reasons:
- to understand what sklearn is doing underneath the hood
- to create something that will work with a fastai TextList

In [ ]:
movie_reviews.vocab.itos[1568]

### Form the embedding vectors for the movie_reviews in the training set and stack them vertically

In [ ]:
# function to build the full document-term matrix
print(f'There are {nDocs} reviews and {nTerms} unique tokens')

In [ ]:
# Define a function to build the full document-term matrix
print(f'there are {nDocs} reviews, and {nTerms} unique tokens in the vocabulary')
def make_full_doc_term_matrix(countVectorizer,n_terms=nTerms,n_docs=nDocs):
    
    # loop through the movie reviews
    for doc_index in range(nDocs):
        
        # make the embedding vector for the current review
        embedding_vector = countVectorizer(doc_index,nTerms)    
            
        # append the embedding vector to the document-term matrix
        if(doc_index == 0):
            A = embedding_vector
        else:
            A = np.vstack((A,embedding_vector))
            
    # return the document-term matrix
    return A

# Build the full document term matrix for the movie_reviews training set
A = make_full_doc_term_matrix(countVectorizer)

In [ ]:
A.shape

# New section

### Explore the `sparsity` of the document-term matrix
The sparsity of a matrix is defined as the fraction of of zero-valued elements

In [ ]:
NNZ = np.count_nonzero(A)
sparsity = (A.size-NNZ)/A.size
print(f'Only {NNZ} of the {A.size} elements in the document-term matrix are nonzero')
print(f'The sparsity of the document-term matrix is {sparsity}')

In [ ]:
fig = plt.figure()
plt.spy(A, markersize=.10, aspect='auto')
fig.set_size_inches(8,6)
fig.savefig('doc_term_matrix.png', dpi = 800)

#### Several observations stand out:
1. Evidently, the document-term matrix is `sparse` ie. has a high proportion of zeros! 
2. The density of the matrix increases toward the `left` edge. This makes sense because the tokens are ordered by usage frequency, with frequency increasing toward the `left`.
3. There is a perplexing pattern of curved vertical `density ripples`. If anyone has an explanation, please let me know! 

#### Next we'll see how to  exploit matrix sparsity to save memory storage space, and compute time and resources.